In [34]:
# Basketball Court Line & Team Detection
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video

In [35]:
# Load video
cap = cv2.VideoCapture('images/basketball_video.mp4')

# Read first frame
ret, frame1 = cap.read()
if not ret:
    raise ValueError("Failed to read video")

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Create video writer
out = cv2.VideoWriter('output/basketball_analysis.avi',cv2.VideoWriter_fourcc(*'XVID'),fps,(frame_width, frame_height))

In [37]:
# Preprocess first frame
gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
gray1 = cv2.GaussianBlur(gray1, (5, 5), 0)

while cap.isOpened():
    ret, frame2 = cap.read()
    if not ret:
        break

    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.GaussianBlur(gray2, (5, 5), 0)

    # Edge detection for line detection
    edge = cv2.Canny(gray2, 50, 150, apertureSize=3)

    # Define ROI (exclude upper part due to camera movement)
    height, width = gray2.shape
    roi_mask = np.zeros_like(gray2)
    court_roi = np.array([[
        (0, int(height * 0.65)),
        (width, int(height * 0.65)),
        (width, height),
        (0, height)
    ]], dtype=np.int32)
    cv2.fillPoly(roi_mask, court_roi, 255)
    masked_edges = cv2.bitwise_and(edge, roi_mask)

    # Hough Line Detection
    lines = cv2.HoughLinesP(masked_edges, 1, np.pi / 180, threshold=100, minLineLength=50, maxLineGap=10)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame2, (x1, y1), (x2, y2), (0, 255, 0), 1)

    # Frame differencing for motion detection
    delta = cv2.absdiff(gray1, gray2)
    thresh = cv2.threshold(delta, 25, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Team color detection (red vs blue)
    hsv = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)
    red_lower = np.array([0, 100, 100])
    red_upper = np.array([10, 255, 255])
    blue_lower = np.array([100, 100, 100])
    blue_upper = np.array([130, 255, 255])

    for contour in contours:
        if cv2.contourArea(contour) < 300:
            continue

        x, y, w, h = cv2.boundingRect(contour)
        if y + h < int(height * 0.4):
            continue

        roi_hsv = hsv[y:y+h, x:x+w]
        mask_red = cv2.inRange(roi_hsv, red_lower, red_upper)
        mask_blue = cv2.inRange(roi_hsv, blue_lower, blue_upper)

        red_count = cv2.countNonZero(mask_red)
        blue_count = cv2.countNonZero(mask_blue)

        if red_count > blue_count and red_count > 50:
            color = (0, 0, 255)
            label = "Team Red"
        elif blue_count > red_count and blue_count > 50:
            color = (255, 0, 0)
            label = "Team Blue"
        else:
            continue

        cv2.rectangle(frame2, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame2, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Write to output video and update frame
    out.write(frame2)
    # Display the processed frame using imshow
    cv2.imshow('Basketball Game Analysis', frame2)

    gray1 = gray2

    # Exit loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and writer
cap.release()
out.release()
cv2.destroyAllWindows()